# 速度指令による経路への追従

numpyのインストールを行うコマンドが入っています。一度実行した後はこの処理はスキップ可能です。初回インストール後にkernelの再起動が必要です。

In [ ]:
!pip install -q numpy

In [ ]:
%run setup_demo_env.py

In [ ]:
import kachaka_api

client = kachaka_api.aio.KachakaApiClient()

ロボットの現在位置を始点とした楕円の経路に沿って移動

In [ ]:
import asyncio
import math
from typing import NamedTuple, Tuple

import numpy as np
from kachaka_api import geometry_util
from kachaka_api.generated.kachaka_api_pb2 import RosOdometry


class LineSegment(NamedTuple):
    start: np.ndarray
    end: np.ndarray

    def calc_distance(self, point: np.ndarray) -> Tuple[float, float]:
        # y 軸は線分と直角の方向
        y_from_line = np.cross(
            point - self.start, self.end - self.start
        ) / np.linalg.norm(self.end - self.start)
        if np.dot(self.end - self.start, point - self.start) < 0:
            distance_from_line = np.linalg.norm(point - self.start)
        elif np.dot(self.start - self.end, point - self.end) < 0:
            distance_from_line = np.linalg.norm(point - self.end)
        else:
            distance_from_line = abs(y_from_line)
        return distance_from_line, y_from_line

    @property
    def angle(self) -> float:
        return math.atan2(self.end[1] - self.start[1], self.end[0] - self.start[0])


def generate_ellipse_way_points(
    radius_x: float, radius_y: float, point_num: int
) -> np.ndarray:
    theta = np.linspace(0, 2 * np.pi, point_num) - np.pi / 2
    x = radius_x * np.cos(theta)
    y = radius_y * np.sin(theta) + radius_y
    return np.vstack((x, y))


def get_xy_yaw_and_linear_speed(
    odom: RosOdometry,
) -> Tuple[Tuple[float, float, float], Tuple[float, float]]:
    pose = odom.pose.pose
    xy_yaw = (
        pose.position.x,
        pose.position.y,
        geometry_util.calculate_yaw_from_quaternion(pose.orientation),
    )
    return xy_yaw, odom.twist.twist.linear.x


ELLIPSE_RADIUS_X = 0.35
ELLIPSE_RADIUS_Y = 0.25
NUM_WAY_POINTS = 30
FOLLOW_LINEAR_SPEED = 0.5
ANGULAR_SPEED_LIMIT = math.radians(90)
IN_PLACE_TURN_THRESHOLD = math.radians(45)
ANGLE_TO_LINE_GAIN = 0.8
DISTANCE_TO_LINE_GAIN = 0.5

# 現在の姿勢を始点として、楕円形の軌道を生成
xy_yaw, _ = get_xy_yaw_and_linear_speed(await client.get_ros_odometry())
trans_mat = geometry_util.calculate_2d_transform_matrix(*xy_yaw)[:2]
local_way_points = generate_ellipse_way_points(
    ELLIPSE_RADIUS_X, ELLIPSE_RADIUS_Y, NUM_WAY_POINTS
)
local_way_points = np.vstack((local_way_points, np.ones((local_way_points.shape[1]))))
# オドメトリ座標系に変換
odom_based_way_points = (trans_mat @ local_way_points).T
line_segments = [
    LineSegment(start=p0, end=p1)
    for p0, p1 in zip(odom_based_way_points[:-1], odom_based_way_points[1:])
]


async def on_odometry(odom):
    # 現在位置に最も近い線分を探す
    xy_yaw, current_linear_speed = get_xy_yaw_and_linear_speed(odom)
    current_position = np.asarray(xy_yaw[:2])
    distances_and_segments = [
        (segment.calc_distance(current_position), segment) for segment in line_segments
    ]
    (_, y_from_line), nearest_segment = min(
        distances_and_segments, key=lambda x: x[0][0]
    )
    angle_to_line = nearest_segment.angle - xy_yaw[2]
    if abs(angle_to_line) > math.pi:
        angle_to_line -= np.sign(angle_to_line) * 2 * math.pi
    # Stanley Controlで角速度を計算
    angular_speed = (
        ANGLE_TO_LINE_GAIN * angle_to_line
        + DISTANCE_TO_LINE_GAIN
        * math.atan2(y_from_line, abs(current_linear_speed) + 1e-3)
    )
    linear_speed = FOLLOW_LINEAR_SPEED
    if abs(angular_speed) > IN_PLACE_TURN_THRESHOLD:
        linear_speed = 0.0
    angular_speed = np.clip(angular_speed, -ANGULAR_SPEED_LIMIT, ANGULAR_SPEED_LIMIT)
    await client.set_robot_velocity(linear_speed, angular_speed)


try:
    await client.set_manual_control_enabled(True)
    client.set_ros_odometry_callback(on_odometry)
    await asyncio.sleep(30)
finally:
    client.set_ros_odometry_callback(None)
    await client.set_robot_velocity(0, 0)
    await client.set_manual_control_enabled(False)